In [1]:
import random
import csv
import pandas as pd
import numpy as np

In [2]:
# train seti için kurallar oluşturuluyor ve rastgele bir train seti oluşturuluyor
def random_veri_uretme(sat_say):
    yas = list(range(18, 66))
    cinsiyet = [0, 1]
    saglik = [0, 1]
    is_durumu = list(range(0, 4))
    ek_gelir = list(range(0, 3))
    maas = list(range(0, 1501))
    borc = [0, 1]
    miras = [0, 1]
    ev = [0, 1]
    araba = [0, 1]
    kredi = [0, 1]

    with open('train.csv', mode="w", newline='') as dosyam:
        yazici = csv.writer(dosyam)
        yazici.writerow(["Yas", "Cinsiyet", "Saglik", "Is Durumu", "Ek Gelir", "Maas", "Borc", "Miras", "Ev", "Araba", "Kredi"])

        tum_veriler = []
        for i in range(sat_say):
            yas_secim = random.choice(yas)
            cinsiyet_secim = random.choice(cinsiyet)
            saglik_secim = random.choice(saglik)
            is_durumu_secim = random.choice(is_durumu)
            ek_gelir_secim = random.choice(ek_gelir)
            maas_secim = random.choice(maas)
            borc_secim = random.choice(borc)
            miras_secim = random.choice(miras)
            ev_secim = random.choice(ev)
            araba_secim = random.choice(araba)
            kredi_secim = random.choice(kredi)

            veri_listesi = [yas_secim, cinsiyet_secim, saglik_secim, is_durumu_secim, ek_gelir_secim, maas_secim,
                            borc_secim, miras_secim, ev_secim, araba_secim, kredi_secim]
            tum_veriler.append(veri_listesi)

        yazici.writerows(tum_veriler)

    print("Veri dosyası başarıyla oluşturuldu.")

In [3]:
# Kullaınıcı kendince bir train seti sayısı belirliyor
satir_sayisi = int(input("Kaç satır veri üretmek istiyorsunuz? "))
random_veri_uretme(satir_sayisi)

Kaç satır veri üretmek istiyorsunuz? 600
Veri dosyası başarıyla oluşturuldu.


In [4]:
df = pd.read_csv('train.csv')
df.head()

,Yas,Cinsiyet,Saglik,Is Durumu,Ek Gelir,Maas,Borc,Miras,Ev,Araba,Kredi
0,29,1,1,2,1,1317,0,1,0,1,1
1,33,0,1,1,0,743,1,0,1,0,1
2,43,1,1,1,1,624,1,0,1,1,0
3,50,0,1,0,0,421,0,1,1,1,1
4,53,0,1,1,1,1197,1,0,1,1,1


In [5]:
print(df.columns)

Index(['Yas', 'Cinsiyet', 'Saglik', 'Is Durumu', 'Ek Gelir', 'Maas', 'Borc',
       'Miras', 'Ev', 'Araba', 'Kredi'],
      dtype='object')


In [6]:
# Gini impurity hesaplaması yapılıyor
def calculate_gini(y):
    p1 = sum(y) / len(y)
    p0 = 1 - p1
    gini = 1 - (p0**2 + p1**2)
    return gini

In [7]:
# Veriler bölünüyor ve verilerde en iyi özellik seçiliyor
def find_best_split(data, target_column):
    features = data.columns.difference([target_column])
    best_gini = float('inf')
    best_feature = None
    best_threshold = None

    for feature in features:
        thresholds = data[feature].unique()
        for threshold in thresholds:
            left_indices = data[data[feature] <= threshold].index
            right_indices = data[data[feature] > threshold].index

            if len(left_indices) == 0 or len(right_indices) == 0:
                # Alt küme boşsa işlem atlanıyor
                continue

            left_gini = calculate_gini(data.loc[left_indices, target_column])
            right_gini = calculate_gini(data.loc[right_indices, target_column])

            gini = (len(left_indices) * left_gini + len(right_indices) * right_gini) / len(data)

            if gini < best_gini:
                best_gini = gini
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold

In [8]:
# Karar ağacını oluşturuluyor
def build_decision_tree(data, target_column):
    # Veri setindeki tek bir sınıf varsa ya da veri seti boşsa
    if len(data[target_column].unique()) == 1 or len(data) == 0:
        return {'prediction': data[target_column].mode().iloc[0]}

    # En iyi özellik ve eşik değeri bulunuyor
    best_feature, best_threshold = find_best_split(data, target_column)

    # En iyi özellik ve eşik değeriyle veri bölünüyor
    left_data = data[data[best_feature] <= best_threshold]
    right_data = data[data[best_feature] > best_threshold]

    # Yeni düğümler oluşturuluyor
    decision_tree = {
        'feature': best_feature,
        'threshold': best_threshold,
        'left': build_decision_tree(left_data, target_column),
        'right': build_decision_tree(right_data, target_column)
    }

    return decision_tree

In [9]:
# Karar ağacını oluşturuluyor
target_column = 'Kredi'  # Tahmin yapılacak hedef sütun belirleniyor
decision_tree = build_decision_tree(df, target_column)

In [10]:
# test veriler için test.csv seti yükleniyor
test_file_path = 'test.csv'
test_df = pd.read_csv(test_file_path, encoding='ISO-8859-1')

In [11]:
# test verileri gösteriliyor
print("Test Verileri:")
print(test_df)

Test Verileri:
   Yas  Cinsiyet  Saglik  Is Durumu  Ek Gelir  Maas  Borc  Miras  Ev  Araba
0   24         0       0          3         1  1000     1      1   1      0
1   38         1       1          1         0   500     0      1   0      1
2   34         1       0          0         2     0     0      0   1      1
3   44         0       1          1         2   150     1      0   1      1
4   23         0       1          3         0   100     0      0   0      1
5   56         0       0          0         1   900     1      1   1      0
6   45         1       0          2         2     0     1      0   1      0
7   60         1       1          2         0   850     0      1   0      1
8   71         1       0          2         1   700     0      0   1      1
9   18         1       0          3         2   420     1      0   1      0


In [12]:
# Karar ağacında tahmin yapılıyor
def predict_tree(tree, sample):
    if 'prediction' in tree:
        return tree['prediction']

    if sample[tree['feature']] <= tree['threshold']:
        return predict_tree(tree['left'], sample)
    else:
        return predict_tree(tree['right'], sample)

In [13]:
# Her satır için tahmin yapılıyor
predictions = []
for index, row in test_df.iterrows():
    sample_data = row.to_dict()
    prediction = predict_tree(decision_tree, sample_data)
    predictions.append(prediction)

# Tahminleri "Kredi" adlı yeni bir sütun olarak "test_df" DataFrame'e ekleniyor
test_df['Kredi'] = predictions

# "test.csv" dosyasına tahminleri içeren yeni sütunu ekleniyor
test_df.to_csv('test_predictions.csv', index=False)

# Yeni sütunu içeren DataFrame'i ekrana yazdırılıyor
print("\nTest için kredi tahmini:")
print(test_df)


Test için kredi tahmini:
   Yas  Cinsiyet  Saglik  Is Durumu  Ek Gelir  Maas  Borc  Miras  Ev  Araba  \
0   24         0       0          3         1  1000     1      1   1      0   
1   38         1       1          1         0   500     0      1   0      1   
2   34         1       0          0         2     0     0      0   1      1   
3   44         0       1          1         2   150     1      0   1      1   
4   23         0       1          3         0   100     0      0   0      1   
5   56         0       0          0         1   900     1      1   1      0   
6   45         1       0          2         2     0     1      0   1      0   
7   60         1       1          2         0   850     0      1   0      1   
8   71         1       0          2         1   700     0      0   1      1   
9   18         1       0          3         2   420     1      0   1      0   

   Kredi  
0      1  
1      0  
2      0  
3      0  
4      1  
5      1  
6      0  
7      0  
8    

In [ ]:
#Test Dosyası oluşturulmadan, elle test verisi girildiğinde

In [14]:
def predict(tree, sample):
    if 'prediction' in tree:
        return tree['prediction']
    
    if sample[tree['feature']] <= tree['threshold']:
        return predict(tree['left'], sample)
    else:
        return predict(tree['right'], sample)

In [15]:
sample_data = {'Yas': 25, 'Cinsiyet': 0, 'Saglik': 1, 'Is Durumu': 0, 'Ek Gelir': 0,
               'Maas': 1200, 'Borc': 0, 'Miras': 1, 'Ev': 0, 'Araba': 1}

# Örnek veri içindeki özellik adlarını karar ağacı modeli tarafından kullanılan adlarla doğrudan eşleştiriliyor
sample_data_for_prediction = {'Yas': sample_data['Yas'], 'Cinsiyet': sample_data['Cinsiyet'],
                               'Saglik': sample_data['Saglik'], 'Is Durumu': sample_data['Is Durumu'],
                               'Ek Gelir': sample_data['Ek Gelir'], 'Maas': sample_data['Maas'],
                               'Borc': sample_data['Borc'], 'Miras': sample_data['Miras'],
                               'Ev': sample_data['Ev'], 'Araba': sample_data['Araba']}

prediction = predict(decision_tree, sample_data_for_prediction)
print(f'Kredi Tahmini: {prediction}')

Kredi Tahmini: 0
